# Online trainings completion rate analysis

## Construction of predictive model to estimate the probability that an employee completes a training

We have already cleaned our dataset in the previous exploratory analysis and data wrangling. The cleaned dataset is constituted of an output variable *training_completed*, two categorical variables (*business_division* and *department*) and 10 numerical variables (2 continuous and 8 discrete).  
We load the dataset and have a look at the data to check everything has been correctly imported.

In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df_clean = pd.read_csv('data/clean/employee_clean.csv')

In [2]:
df_clean.head()

,training_completed,business_division,department,engagement_score,tenure,leadership_score,overtime,incidents,duration_elearning,time_in_title,delta_trainings_last_year,risk_of_leaving,rating
0,1,WM,GHL,3,16,6.125277,29,1,113,15,4,0.632862,1.0
1,1,WM,GHL,4,11,5.631186,41,1,91,21,4,0.422677,2.0
2,1,SB,GLI,4,9,5.951434,41,2,136,6,5,0.634727,1.0
3,1,SB,GHL,2,12,6.901432,47,2,99,10,5,0.594129,2.0
4,1,IB,AML,3,10,5.038099,47,1,142,15,5,0.585208,3.0


Two main points need to be addressed before moving to the model definition:
1. The two categorical variables *business_division* and *department* need to be encoded as numerical variables.
2. The missing values in the variable *rating* need to be handled.

### Encoding of categorical variables *business_division* and *department*

For the first point, since there is no natural ordinal relationship between the values of the variables, we proced with a One-Hot Encoding of each variable.

In [3]:
# one-hot encoding of business_division and department
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# define one-hot encoder
transformer = make_column_transformer(
    (OneHotEncoder(), ["business_division", "department"]),
    remainder="passthrough")

# fit one-hot encoder and transform the two columns with the fitted encoder
transformed_columns = transformer.fit_transform(df_clean)

# create new dataframe df with encoded features
df = pd.DataFrame(transformed_columns, columns=transformer.get_feature_names())

df.head()

,onehotencoder__x0_AM,onehotencoder__x0_IB,onehotencoder__x0_SB,onehotencoder__x0_WM,onehotencoder__x1_ABD,onehotencoder__x1_ADC,onehotencoder__x1_AML,onehotencoder__x1_GHL,onehotencoder__x1_GLI,onehotencoder__x1_HDO,...,engagement_score,tenure,leadership_score,overtime,incidents,duration_elearning,time_in_title,delta_trainings_last_year,risk_of_leaving,rating
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,3.0,16.0,6.125277,29.0,1.0,113.0,15.0,4.0,0.632862,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,4.0,11.0,5.631186,41.0,1.0,91.0,21.0,4.0,0.422677,2.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,4.0,9.0,5.951434,41.0,2.0,136.0,6.0,5.0,0.634727,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,2.0,12.0,6.901432,47.0,2.0,99.0,10.0,5.0,0.594129,2.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.0,10.0,5.038099,47.0,1.0,142.0,15.0,5.0,0.585208,3.0


### Handling missing values in *rating*
For the second point, there are several possibilities, from deleting the rows with missing values to deleting the column to filling the missing value with specific values. 

We exclude the first approach since we would loose half of the dataset. 

The third approach would be the best to preserve all the available information, whereas since the variable represents a scale with a clear ordinal order between the values, setting any value would introduce a fictitious assumption on the rating value for the observations with missing values. 

Setting a value higher than the maximum would force the assumptions that the employees with no data on the performance evaluation are better performer that all the others, on the opposite a lower value (e.g. 0, -1, -99) would force introduce the opposite bias. We could set a value dependent on another feature with high correlation with the rating, yet as we have seen in the exploratory analysis, the variable is mostly uncorrelated with all the others. Hence, the safest approach is to drop the column altogether. Doing this we loose some potentially useful information, but we avoid introducing an external bias in the model, forcing a priori assumptions, something critical when developing a prediction model on personal data.

In [4]:
# drop column rating
df.drop("rating", axis=1, inplace=True)
df.head()

,onehotencoder__x0_AM,onehotencoder__x0_IB,onehotencoder__x0_SB,onehotencoder__x0_WM,onehotencoder__x1_ABD,onehotencoder__x1_ADC,onehotencoder__x1_AML,onehotencoder__x1_GHL,onehotencoder__x1_GLI,onehotencoder__x1_HDO,...,training_completed,engagement_score,tenure,leadership_score,overtime,incidents,duration_elearning,time_in_title,delta_trainings_last_year,risk_of_leaving
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,3.0,16.0,6.125277,29.0,1.0,113.0,15.0,4.0,0.632862
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,4.0,11.0,5.631186,41.0,1.0,91.0,21.0,4.0,0.422677
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,4.0,9.0,5.951434,41.0,2.0,136.0,6.0,5.0,0.634727
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,2.0,12.0,6.901432,47.0,2.0,99.0,10.0,5.0,0.594129
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,3.0,10.0,5.038099,47.0,1.0,142.0,15.0,5.0,0.585208


### Features pairplot 
As a useful tool to explore the dependency structure between the numerical features, we compute the features pairplot. To do that, we use the dataset before the encoding of the categorical variables and we exclude the two categorical variables.

In [5]:
import seaborn as sns

df_pairplot = df_clean.drop(["business_division", "department", "rating"], axis=1)

In [6]:
# sns.pairplot(df_pairplot, hue="training_completed", diag_kind="hist").fig.set_size_inches(15,15)

## Model construction

A wide variety of machine learning models are available to tackle a classification problem like the one we are addressing. Here we report a non-comprehensive list of the models we have considered.

1. Logistic regression
2. K-nearest neighbours
3. Naive Bayes Classifier
4. Support Vector Machine
5. Decision Tree
6. Random Forest

To indentify the best model for our task, we have to consider two main points:

1. Model output in the form of a probability
2. Model interpretability

In particular, for the second point we have to deal with the possible trade-off between model accuracy and model interpretability (excluding neural networks for example). In the following, we will train and test all the six models, leveraging on the efficiency of scikit-learn in the easy prototyping machine learning models. We will compare the performance of the models and then based on considerations on both performance and interpretability we will focus on one of them, deepening its analysis.

### Create train-test split

First we store the features (input variables) in the dataframe X and the output variable (*training_completed*) in the dataframe y. Then we apply the train-test split with the default scikit-learn percentages 75%/25% train-test.

In [7]:
X = df.loc[:, df.columns != "training_completed"]
y = df["training_completed"]

from sklearn.model_selection import train_test_split
# 75/25 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [8]:
X_train.head()   

,onehotencoder__x0_AM,onehotencoder__x0_IB,onehotencoder__x0_SB,onehotencoder__x0_WM,onehotencoder__x1_ABD,onehotencoder__x1_ADC,onehotencoder__x1_AML,onehotencoder__x1_GHL,onehotencoder__x1_GLI,onehotencoder__x1_HDO,...,onehotencoder__x1_ZEF,engagement_score,tenure,leadership_score,overtime,incidents,duration_elearning,time_in_title,delta_trainings_last_year,risk_of_leaving
4154,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,13.0,4.711069,36.0,1.0,115.0,18.0,4.0,0.436294
4820,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,4.0,8.0,6.701531,44.0,2.0,102.0,11.0,5.0,0.530892
1202,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,2.0,14.0,5.396495,47.0,2.0,119.0,14.0,5.0,0.527053
3759,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,10.0,5.693930,47.0,1.0,106.0,13.0,3.0,0.388433
622,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,4.0,7.0,6.773361,56.0,2.0,73.0,7.0,6.0,0.475292


In [9]:
X_test.head()

,onehotencoder__x0_AM,onehotencoder__x0_IB,onehotencoder__x0_SB,onehotencoder__x0_WM,onehotencoder__x1_ABD,onehotencoder__x1_ADC,onehotencoder__x1_AML,onehotencoder__x1_GHL,onehotencoder__x1_GLI,onehotencoder__x1_HDO,...,onehotencoder__x1_ZEF,engagement_score,tenure,leadership_score,overtime,incidents,duration_elearning,time_in_title,delta_trainings_last_year,risk_of_leaving
398,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,9.0,6.041170,28.0,1.0,179.0,10.0,3.0,0.558857
3833,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,4.0,11.0,5.829723,52.0,1.0,76.0,13.0,3.0,0.364687
4836,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,11.0,6.961637,41.0,2.0,84.0,11.0,5.0,0.517792
4572,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,3.0,9.0,4.487510,51.0,1.0,167.0,23.0,5.0,0.480742
636,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,13.0,5.300257,46.0,1.0,116.0,20.0,4.0,0.480431


In [10]:
y_train.head()

4154    1.0
4820    1.0
1202    0.0
3759    1.0
622     1.0
Name: training_completed, dtype: float64

In [11]:
y_test.head()

398     0.0
3833    1.0
4836    1.0
4572    0.0
636     1.0
Name: training_completed, dtype: float64

## Features scaling
Even if several of the models under investigation are not sensitive to the features scaling, the K-nearest neighbours performance is deeply affected by the scaling of the features, as well as the logistic regression and support vector classifier due to the application of regularization.

Moreover, the features scaling is important to have a straightforward comparison of the coefficient of logistic regression and linear support vector classifier. Hence, we procede with the scaling of the numerical features of the problem. We scale also the encoded categorical variables, since the model will not respond differently to columns of one-hot-encoded variables than to numerical variables. Hence, they should be treated similarly.

It is extremely important that the scaler is fitted only on the training data, to avoid data leakege issues where the training phase leverage on information from the test data, hence invalidating or making less effective the generalization performance testing. Moreover, the same scaling must be applied to both training and test data.

In [12]:
features_names = X_train.columns
features_names

Index(['onehotencoder__x0_AM', 'onehotencoder__x0_IB', 'onehotencoder__x0_SB',
       'onehotencoder__x0_WM', 'onehotencoder__x1_ABD',
       'onehotencoder__x1_ADC', 'onehotencoder__x1_AML',
       'onehotencoder__x1_GHL', 'onehotencoder__x1_GLI',
       'onehotencoder__x1_HDO', 'onehotencoder__x1_NAP',
       'onehotencoder__x1_POR', 'onehotencoder__x1_QHA',
       'onehotencoder__x1_ZEF', 'engagement_score', 'tenure',
       'leadership_score', 'overtime', 'incidents', 'duration_elearning',
       'time_in_title', 'delta_trainings_last_year', 'risk_of_leaving'],
      dtype='object')

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# we can safely suppress the warning we get due to chained assignment
pd.options.mode.chained_assignment = None  # default='warn'

X_train[features_names] = scaler.fit_transform(X_train[features_names])
X_test[features_names] = scaler.transform(X_test[features_names])

X_train.head()                                          

,onehotencoder__x0_AM,onehotencoder__x0_IB,onehotencoder__x0_SB,onehotencoder__x0_WM,onehotencoder__x1_ABD,onehotencoder__x1_ADC,onehotencoder__x1_AML,onehotencoder__x1_GHL,onehotencoder__x1_GLI,onehotencoder__x1_HDO,...,onehotencoder__x1_ZEF,engagement_score,tenure,leadership_score,overtime,incidents,duration_elearning,time_in_title,delta_trainings_last_year,risk_of_leaving
4154,-0.552911,1.733900,-0.582893,-0.596857,2.943628,-0.330859,-0.344095,-0.326376,-0.340693,-0.336288,...,-0.326876,-0.217112,0.959989,-0.730021,-0.511394,-0.712842,-0.058142,0.675290,-0.565535,-0.579793
4820,-0.552911,-0.576734,-0.582893,1.675442,-0.339717,-0.330859,-0.344095,-0.326376,-0.340693,2.973637,...,-0.326876,1.274041,-1.058151,0.764728,0.171041,1.297051,-0.417158,-0.783791,0.316276,0.104112
1202,-0.552911,-0.576734,1.715581,-0.596857,-0.339717,3.022439,-0.344095,-0.326376,-0.340693,-0.336288,...,-0.326876,-1.708265,1.363617,-0.215296,0.426954,1.297051,0.052324,-0.158470,0.316276,0.076361
3759,1.808610,-0.576734,-0.582893,-0.596857,-0.339717,-0.330859,-0.344095,-0.326376,-0.340693,-0.336288,...,-0.326876,1.274041,-0.250895,0.008064,0.426954,-0.712842,-0.306692,-0.366910,-1.447345,-0.925807
622,-0.552911,1.733900,-0.582893,-0.596857,-0.339717,-0.330859,-0.344095,3.063954,-0.340693,-0.336288,...,-0.326876,1.274041,-1.461779,0.818669,1.194692,1.297051,-1.218039,-1.617551,1.198087,-0.297848


We now procede in training and testing the six models, computing several evaluation metrics to compare their performances. The goal of this step is to gain insights into the best performing models, to then select one of them, in light also of model interpretability, and then detailing the analysis for the chosen model.

## Dummy classifier
We first train and test a dummy classifier to have a benchmark for the performances of the other models.

In [14]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)

y_predicted = dummy.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted, target_names=['0', '1']))

# clearly since the classifier is constantly predicting the most frequent class, 
# the metrics for the other class are ill-defined, this motivates the warning we get

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       373
           1       0.70      1.00      0.82       877

    accuracy                           0.70      1250
   macro avg       0.35      0.50      0.41      1250
weighted avg       0.49      0.70      0.58      1250



/home/davide/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/davide/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/davide/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Logistic regression

In [15]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression().fit(X_train, y_train)

In [16]:
y_predicted = logreg.predict(X_test)
print(classification_report(y_test, y_predicted, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.82      0.73      0.77       373
           1       0.89      0.93      0.91       877

    accuracy                           0.87      1250
   macro avg       0.85      0.83      0.84      1250
weighted avg       0.87      0.87      0.87      1250



## Linear Support Vector Classifier

In [17]:
from sklearn.svm import LinearSVC

linearsvc = LinearSVC().fit(X_train, y_train)

/home/davide/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [18]:
y_predicted = linearsvc.predict(X_test)
print(classification_report(y_test, y_predicted, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.82      0.73      0.77       373
           1       0.89      0.93      0.91       877

    accuracy                           0.87      1250
   macro avg       0.85      0.83      0.84      1250
weighted avg       0.87      0.87      0.87      1250



## K-nearest neighbours

In [19]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = round(np.sqrt(X_train.shape[0]))).fit(X_train, y_train)

In [20]:
y_predicted = knn.predict(X_test)
print(classification_report(y_test, y_predicted, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.96      0.27      0.42       373
           1       0.76      1.00      0.86       877

    accuracy                           0.78      1250
   macro avg       0.86      0.63      0.64      1250
weighted avg       0.82      0.78      0.73      1250



## Naive Bayes Classifier

In [21]:
from sklearn.naive_bayes import GaussianNB

nbclf = GaussianNB().fit(X_train, y_train)

In [22]:
y_predicted = nbclf.predict(X_test)
print(classification_report(y_test, y_predicted, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.78      0.66      0.72       373
           1       0.87      0.92      0.89       877

    accuracy                           0.84      1250
   macro avg       0.82      0.79      0.80      1250
weighted avg       0.84      0.84      0.84      1250



## Decision Tree

In [23]:
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier(random_state = 0).fit(X_train, y_train)

In [24]:
y_predicted = decisiontree.predict(X_test)
print(classification_report(y_test, y_predicted, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.75      0.78      0.76       373
           1       0.90      0.89      0.90       877

    accuracy                           0.86      1250
   macro avg       0.83      0.83      0.83      1250
weighted avg       0.86      0.86      0.86      1250



## Random Forest

In [25]:
from sklearn.ensemble import RandomForestClassifier

rndforest = RandomForestClassifier(random_state = 0).fit(X_train, y_train)

In [26]:
y_predicted = rndforest.predict(X_test)
print(classification_report(y_test, y_predicted, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.89      0.79      0.84       373
           1       0.92      0.96      0.94       877

    accuracy                           0.91      1250
   macro avg       0.90      0.88      0.89      1250
weighted avg       0.91      0.91      0.91      1250



Comparing the performance of the models, we observe that the best performance metrics are achieved by the random forest model. Also the decision tree and linear support vector classifier show good performance. A special mention goes to the naive bayes classifier that despite its simplicitly and its strong assumptions on the data distribution, achieve a performance comparable to more complex models. This is probably due to the low correlation between the different features that we have observed on the exploratory analysis, together with the fact that the features distributions are all well approximated by normal distributions. Both conditions are favourable for the performance of the model.

Finally, we observe a good performance of the logistic regression model, comparable to the best models in this analysis and only lower to the random forest, a more complex model.

Our choice falls on the regression model, both for the good performance already mentioned but most importantly because it matches the two requirements of model output in the form of a probability and model interpretability.

For the first point, the output of the model can be directly interpreted as the probability an employee completes a training. 

In [27]:
logreg.predict(X_test.head(1))

array([0.])

In [28]:
logreg.predict_proba(X_test.head(1))

array([[0.68534646, 0.31465354]])

The model is also very good in terms of interpretability, indeed due to the linearity of the model the fitted coefficients can be directly interpreted as a measure of the importance of the associated feature.

In [29]:
coefficients = pd.DataFrame(list(X_train.columns)).copy()
coefficients.insert(len(coefficients.columns),"coefficient value",logreg.coef_.transpose())
coefficients.columns = ["associated feature","coefficient value"]
coefficients["abs coefficient value"] = abs(coefficients["coefficient value"])
coefficients = coefficients.sort_values(by=["abs coefficient value"], ascending=False)
coefficients.drop("abs coefficient value", axis=1, inplace=True)

coefficients

,associated feature,coefficient value
19,duration_elearning,-2.906406
22,risk_of_leaving,1.202586
21,delta_trainings_last_year,-1.186558
16,leadership_score,-0.648950
20,time_in_title,0.607438
14,engagement_score,0.560371
15,tenure,-0.379290
18,incidents,-0.336190
17,overtime,0.126781
2,onehotencoder__x0_SB,-0.111802


Since all the input variables have been scaled to have zero mean and unit standard deviations, the coefficients can provide the basis for a crude feature importance score. In particular, the coefficient with largest absolute values are the one impacting the more the model predicition, due to the linearity of the model. 

We can infer that the following are top 5 drivers of the estimates.

In [30]:
coefficients.head()

,associated feature,coefficient value
19,duration_elearning,-2.906406
22,risk_of_leaving,1.202586
21,delta_trainings_last_year,-1.186558
16,leadership_score,-0.648950
20,time_in_title,0.607438


1. The duration_elearning is the most relevant variable, with the largest negative impact on the prediction. We can interpret it as the longer is the duration of the training the less probable is the employee will complete it, as can be expected.

2. Then we observe that the risk_of_leaving impact positively the prediction the employee will complete the training, probably staff with the intention to resign is more prone to learn new skills to be more competitive in the job market and less prone to invest the working time in producing value for the organization they want to leave.

3. The variable delta_trainings_last_year impacts negatively on the probability of training completion, we can give the following interpretation. The more training hours have not been completed last year by the employee the more is probable the employee will not complete the training this year.

4. The leadership score also impacts negatively on the training completion, most probably due to a lack of time or the mismatch of training offer with respect to the managerial skills needed by the management.

5. Finally, the time in title is the fifth driving factor, this can be interpreted as the willingness of the employee to evolve and improve its skills when in a stable position, or in light of running for a future possible promotion.

In [31]:
logreg.feature_importances_

AttributeError: 'LogisticRegression' object has no attribute 'feature_importances_'